In [1]:
import pandas as pd
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.utils import class_weight
import multiprocessing as mp
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
import parmap

In [2]:
topics_index_to_name_map = {
    0: 'Agriculture, animals, food and rural affairs',
    1: 'Asylum, immigration and nationality',
    2: 'Business, industry and consumers',
    3: 'Communities and families',
    4: 'Crime, civil law, justice and rights',
    5: 'Culture, media and sport',
    6: 'Defence',
    7: 'Economy and finance',
    8: 'Education',
    9: 'Employment and training',
    10: 'Energy and environment',
    11: 'European Union',
    12: 'Health services and medicine',
    13: 'Housing and planning',
    14: 'International affairs',
    15: 'Parliament, government and politics',
    16: 'Science and technology',
    17: 'Social security and pensions',
    18: 'Social services',
    19: 'Transport',
    20: 'Others'
}
topics_name_to_index_map = {y:x for x,y in topics_index_to_name_map.items()}

def strip_short2(text):
    return strip_short(text, minsize=4)


def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)

def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return topics_name_to_index_map[t]
        
    return topics_name_to_index_map[topic]

In [11]:
years = [2014, 2015, 2016, 2017, 2018]
rows = []

for year in years:
    df = df = pd.read_csv('../data/news_predictions/news_{}_predictions.csv'.format(year))
    df = df.drop(['Unnamed: 0'], axis=1)
    df = df.loc[df.top1_acc >= 40]
    df = df[['source', 'month', 'year']]
    df['count'] = 1
    df_group = df.groupby(['source', 'month', 'year'], as_index=False).sum()
    df_group = df_group.sort_values(by=['year', 'month'])
    for index, row in df_group.iterrows():
        rows.append([row['year'], row['month'], row['source'], row['count']])
    display(df_group)
    
res_df = pd.DataFrame(rows, columns=['year', 'month', 'source', 'count'])

,source,month,year,count
0,Belfast Telegraph,1,2014,6108
12,Express,1,2014,5455
24,Guardian,1,2014,5140
36,Herald,1,2014,2116
48,Independent,1,2014,7606
...,...,...,...,...
167,Standard,12,2014,1812
179,Star,12,2014,2830
191,Sun,12,2014,329
203,Telegraph,12,2014,9076


,source,month,year,count
0,Belfast Telegraph,1,2015,3915
12,Express,1,2015,2785
24,Guardian,1,2015,3082
36,Herald,1,2015,1325
48,Independent,1,2015,3947
...,...,...,...,...
167,Standard,12,2015,1766
179,Star,12,2015,2251
191,Sun,12,2015,180
203,Telegraph,12,2015,5070


,source,month,year,count
0,Belfast Telegraph,1,2016,4672
12,Express,1,2016,4944
24,Guardian,1,2016,4884
36,Herald,1,2016,1307
48,Independent,1,2016,5249
...,...,...,...,...
167,Standard,12,2016,1662
179,Star,12,2016,3129
191,Sun,12,2016,132
203,Telegraph,12,2016,5425


,source,month,year,count
0,Belfast Telegraph,1,2017,6892
12,Express,1,2017,11194
24,Guardian,1,2017,2797
36,Herald,1,2017,1803
48,Independent,1,2017,5514
...,...,...,...,...
167,Standard,12,2017,2422
179,Star,12,2017,4758
191,Sun,12,2017,369
203,Telegraph,12,2017,6335


,source,month,year,count
0,Belfast Telegraph,1,2018,6763
12,Express,1,2018,10913
24,Guardian,1,2018,2802
36,Herald,1,2018,1553
48,Independent,1,2018,4946
...,...,...,...,...
167,Standard,12,2018,2504
179,Star,12,2018,4020
191,Sun,12,2018,352
203,Telegraph,12,2018,5637


AttributeError: module 'pandas' has no attribute 'DataFram'

In [13]:
res_df = pd.DataFrame(rows, columns=['year', 'month', 'source', 'count'])
res_df

,year,month,source,count
0,2014,1,Belfast Telegraph,6108
1,2014,1,Express,5455
2,2014,1,Guardian,5140
3,2014,1,Herald,2116
4,2014,1,Independent,7606
...,...,...,...,...
1075,2018,12,Standard,2504
1076,2018,12,Star,4020
1077,2018,12,Sun,352
1078,2018,12,Telegraph,5637


In [14]:
res_df.to_csv('agg_news_count_40_threshold.csv')

In [70]:
year = 2015

In [71]:
df = pd.read_csv('./data/news_predictions/news_{}_predictions.csv'.format(year))

In [72]:
df = df.drop(['Unnamed: 0'], axis=1)

In [73]:
df

,source_id,source,day,month,year,program_name,transcript,parliament,top1_topic,top1_acc,top2_topic,top2_acc,top3_topic,top3_acc
0,163795,Belfast Telegraph,1,1,2015,Plans for site of former factory to be 12-acre...,"The Artnz Belting company,\nwhich had been a m...",1,Transport,48.68,"Parliament, government and politics",12.64,Communities and families,12.03
1,163795,Belfast Telegraph,1,1,2015,Councillors left red-faced over bid for new pr...,"However, it was then revealed that a database ...",1,Communities and families,28.51,Transport,15.06,Others,10.18
2,163795,Belfast Telegraph,1,1,2015,Woman breached bail and spent Christmas in cus...,Sharon Campbell from Lawrence Hill in Londonde...,1,Others,25.44,"Crime, civil law, justice and rights",24.46,"Asylum, immigration and nationality",12.85
3,163795,Belfast Telegraph,1,1,2015,Driver who was three times legal limit jailed,Paul McCloskey (36) of Horace Street in London...,1,"Crime, civil law, justice and rights",35.05,Others,10.45,Defence,8.50
4,163795,Belfast Telegraph,1,1,2015,A oneness that unites us in Christ;\nMY VIEW,That means more than simply recording the numb...,1,"Parliament, government and politics",53.14,International affairs,12.30,"Culture, media and sport",12.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1619661,412338,Wales,31,12,2015,Morning news headlines: Homes evacuated and th...,Hundreds of homes have been evacuated and thou...,1,Others,30.94,"Business, industry and consumers",17.66,"Parliament, government and politics",13.39
1619662,412338,Wales,31,12,2015,How many parents have been fined for taking th...,Figures gathered by WalesOnline reveal a postc...,1,Education,36.93,"Parliament, government and politics",25.48,Communities and families,18.26
1619663,412338,Wales,31,12,2015,Enterprise zone based firms across Wales get £...,"Economy Minister, Edwina Hart announced the aw...",1,"Parliament, government and politics",80.30,"Business, industry and consumers",4.78,"Culture, media and sport",3.82
1619664,412338,Wales,31,12,2015,Changes to Wales' organ donation system will s...,Carwyn Jones was among the politicians to deli...,1,"Parliament, government and politics",90.38,Communities and families,1.62,"Culture, media and sport",1.33


In [61]:
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

topics = ['Agriculture, animals, food and rural affairs', 'Asylum, immigration and nationality', 'Business, industry and consumers', 'Communities and families', 'Crime, civil law, justice and rights', 'Culture, media and sport', 'Defence', 'Economy and finance', 'Education', 'Employment and training', 'Energy and environment', 'European Union', 'Health services and medicine', 'Housing and planning', 'International affairs', 'Parliament, government and politics', 'Science and technology', 'Social security and pensions', 'Social services', 'Transport', 'Others'
]

months_map = {i: months[i-1] for i in range(1, 13)}

In [76]:
ids = [389195, 138620, 138794]

for id_ in ids:
    df2 = df.loc[df.source_id == id_]
    
    df2 = df2.loc[~df.top1_topic.str.contains('Others')]
    df2 = df2.loc[df.top1_acc >= 40]
    
    top1_counts = {month: {topic: 0 for topic in topics} for month in range(1, 13)}
    top2_counts = {month: {topic: 0 for topic in topics} for month in range(1, 13)}
    top3_counts = {month: {topic: 0 for topic in topics} for month in range(1, 13)}
    count_month = {month: 0 for month in range(1, 13)}

    for index, row in df2.iterrows():
        month = row['month']
        count_month[month] += 1
        top1 = row['top1_topic']
        top2 = row['top2_topic']
        top3 = row['top3_topic']
        top1_counts[month][top1] += 1
        top2_counts[month][top2] += 1
        top3_counts[month][top3] += 1

    rows = []
    for month in range(1, 13):
        for topic in topics:
            share1 = top1_counts[month][topic] / count_month[month]
            share2 = top2_counts[month][topic] / count_month[month]
            share3 = top3_counts[month][topic] / count_month[month]
            row = [months_map[month], topic, share1, share2, share3]
            rows.append(row)

    res = pd.DataFrame(rows, columns=['month', 'topic', 'top1', 'top2', 'top3'])

    display(res)

    res.to_csv('{}_news_topic_month_shares_{}_no_Others.csv'.format(id_, year))

,month,topic,top1,top2,top3
0,jan,"Agriculture, animals, food and rural affairs",0.029759,0.017406,0.032004
1,jan,"Asylum, immigration and nationality",0.000000,0.001684,0.014879
2,jan,"Business, industry and consumers",0.110612,0.140090,0.131668
3,jan,Communities and families,0.001684,0.007299,0.010387
4,jan,"Crime, civil law, justice and rights",0.059798,0.043796,0.042953
...,...,...,...,...,...
247,dec,Science and technology,0.000000,0.002869,0.016858
248,dec,Social security and pensions,0.000000,0.000000,0.000359
249,dec,Social services,0.000000,0.000000,0.000000
250,dec,Transport,0.059182,0.020086,0.030846


,month,topic,top1,top2,top3
0,jan,"Agriculture, animals, food and rural affairs",0.025923,0.015625,0.024858
1,jan,"Asylum, immigration and nationality",0.001065,0.003906,0.018466
2,jan,"Business, industry and consumers",0.093040,0.127131,0.150568
3,jan,Communities and families,0.002841,0.007457,0.013494
4,jan,"Crime, civil law, justice and rights",0.073509,0.048295,0.046520
...,...,...,...,...,...
247,dec,Science and technology,0.000000,0.002174,0.013859
248,dec,Social security and pensions,0.000000,0.000815,0.002989
249,dec,Social services,0.000000,0.000000,0.000000
250,dec,Transport,0.029620,0.013587,0.027989


,month,topic,top1,top2,top3
0,jan,"Agriculture, animals, food and rural affairs",0.019270,0.029412,0.057809
1,jan,"Asylum, immigration and nationality",0.000000,0.007099,0.026369
2,jan,"Business, industry and consumers",0.103448,0.130832,0.112576
3,jan,Communities and families,0.000000,0.010142,0.018256
4,jan,"Crime, civil law, justice and rights",0.117647,0.055781,0.045639
...,...,...,...,...,...
247,dec,Science and technology,0.000000,0.002447,0.010604
248,dec,Social security and pensions,0.000000,0.000000,0.000000
249,dec,Social services,0.000000,0.000000,0.000000
250,dec,Transport,0.042414,0.028548,0.036705
